In [ ]:
import zarr
import xarray as xr
import numpy as np
import time
import math
from geopy.distance import geodesic

In [ ]:
st = time.time()
ds = xr.open_zarr(
    'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
ds

In [ ]:
# base=90 for latitude; base=180 for longitude
def gridded_arcsec(x, base=90, arc=3600/15):
    return ((int(x) + base)* arc + math.ceil((x-int(x))*arc))

arc = int(3600/15) #arc
print(gridded_arcsec(0, 180, arc))
print(gridded_arcsec(0, 90, arc))
print(gridded_arcsec(-90, 90, arc))
print(gridded_arcsec(90, 90, arc))
print(gridded_arcsec(-180, 180, arc))
print(gridded_arcsec(180, 180, arc))
print(gridded_arcsec(0.0015, 180, arc))
print(gridded_arcsec(-0.00207, 180, arc))
print(gridded_arcsec(-85.375, 90, arc))
print(gridded_arcsec(48.37918, 90, arc))


In [43]:
arcsec = 15
arc = int(3600/arcsec) #15 arc-second
basex = 180 #-180 - 180 <==> 0 - 360, half is 180
basey = 90  #-90 - 90 <==> 0 - 180, half is 90
halfxidx = 180 * arc   #in netcdf, longitude length = 86400
halfyidx = 90 * arc    #in netcdf, latitude length = 43200
lon = [121.53967066033685,121.5649924414543] 
#test bug (and solved)
#[121.52789789212326,121.58410196573966,121.57161216694406,121.53726523073809,121.53664076385839,121.57723257849847] #[122.80154260534586,122.80154260534586] 
#[123, 123.33, 123.38]
lat = [25.070035818788032,25.06911681954908]
#[25.054700205734235,25.033770970073878,24.993599440972766,24.982847101743943,24.942092803893317,24.930769789455212] 
#[23.782329893520448,23.769188442795453]
subsetFlag = True
zmode = "line"
format = "row"
if (len(lon) != len(lat)):
    print("Input Error: check longitude, latitude length not equal")
    #return
# May not have exact lon, lat in gridded lon-lat, so need to calculate index  
elif (len(lon) == 1):
    st = time.time()
    lon0 = gridded_arcsec(lon[0], basex, arc)
    lat0 = gridded_arcsec(lat[0], basey, arc)
    mlon0x = ds["lon"][lon0]
    mlat0x = ds["lat"][lat0]
    st1 = ds.sel(lon=mlon0x, lat=mlat0x)
    loc1 = [lon[0], lat[0]]
    xt1 = np.array([st1['elevation'].values])
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(xt1)
else:
    st = time.time()
    idx1 = np.empty(shape=[0, 2], dtype=np.int16)
    loc1 = np.empty(shape=[0, 2], dtype=float)
    dis1 = np.array([0.0])
    mlon0 = np.min(lon)
    #mlon1 = np.max(lon)
    mlat0 = np.min(lat)
    #mlat1 = np.max(lat)
    #ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1))
    mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0 #if do subsetting dataset, reference-0-x,y should be biased
    mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
    print(mlonbase, mlatbase)
    #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
    #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
    for i in range(len(lon)-1):
        lonidx0 = gridded_arcsec(lon[i], basex, arc)
        latidx0 = gridded_arcsec(lat[i], basey, arc)
        lonidx1 = gridded_arcsec(lon[i+1], basex, arc)
        latidx1 = gridded_arcsec(lat[i+1], basey, arc)
        #ds_s1 = ds.sel(lon=slice(lon[i], lon[i+1]), lat=slice(lat[i], lat[i+1]))
        #index from gridded_arcsec will change is reference is ds_s1, not ds
        if lonidx0 == lonidx1 and latidx0 == latidx1:
            idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
            if i >= 1:
                dis1 = np.append(dis1, 0.0, axis=None) #to match the same length
        elif zmode == 'point':
            idx1 = np.append(
                idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
            dist = geodesic((lat[i], lon[i]),
                            (lat[i+1], lon[i+1])).km
            dis1 = np.append(dis1, dist, axis=None)
            if i == len(lon)-2:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lon[i+1], lat[i+1]]], axis=0)            
        else:        
            if lon[i] == lon[i+1]:
                stepi = -1 if latidx0 > latidx1 else 1
                for k,y in enumerate(range(latidx0, latidx1, stepi)):
                    idx1 = np.append(idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                    locy0 = y/arc - basey
                    locy0i= int(locy0)
                    doty0i= locy0-locy0i-0.25/arc #a small bias to make sure it's in grid
                    print("yi: ", y, locy0, locy0i,locy0i + doty0i)
                    loc1 = np.append(loc1, [[lon[i], locy0i + doty0i]], axis=0)
                    if i>=1 or k>=1:
                        lk = len(loc1)-1
                        dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                        (loc1[lk, 1], loc1[lk, 0])).km
                        dis1 = np.append(dis1, dist, axis=None)
                        print("dist: ", dist, " at ", loc1[lk-1, 1], " from ", loc1[lk, 1])                    
            elif lat[i] == lat[i+1]:
                stepi = -1 if lonidx0 > lonidx1 else 1
                for k,x in enumerate(range(lonidx0, lonidx1, stepi)):
                    idx1 = np.append(idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                    locx0 = x/arc - basex
                    locx0i= int(locx0)
                    dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                    print("xi: ", x, locx0i + dotx0i)
                    loc1 = np.append(loc1, [[locx0i + dotx0i, lat[i]]], axis=0)
                    if i>=1 or k>=1:
                        lk = len(loc1)-1
                        dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                        (loc1[lk, 1], loc1[lk, 0])).km
                        dis1 = np.append(dis1, dist, axis=None)                    
            else:
                m = (lat[i+1]-lat[i])/(lon[i+1]-lon[i])
                b = lat[i] - m * lon[i] #y = mx + b
                print("m, b:", m ,b)
                stepi = -1 if lonidx0 > lonidx1 else 1
                for k, x in enumerate(range(lonidx0, lonidx1, stepi)):
                    if (k==0): #should consider internal node not repeated twice
                        idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
                        if i >= 1:
                            lk = len(loc1)-1
                            dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                            (loc1[lk, 1], loc1[lk, 0])).km
                            dis1 = np.append(dis1, dist, axis=None)

                    else:
                        locx0 = x/arc - basex
                        locx0i= int(locx0)
                        dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                        locx1 = locx0i + dotx0i 
                        locy1 = m * locx1 + b 
                        y = gridded_arcsec(locy1, basey, arc)
                        idx1 = np.append(idx1, [[y-mlatbase, x-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[locx1, locy1]], axis=0)
                        print("line: ", x, locx0, dotx0i, locx1, locy1, y)
                        lk = len(loc1)-1
                        dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                        (loc1[lk, 1], loc1[lk, 0])).km
                        dis1 = np.append(dis1, dist, axis=None)
                        #print("ith-turn: ", i, k, len(idx1), len(loc1), len(dis1))

            if i == len(lon)-2 and lonidx1 != idx1[len(idx1)-1, 1] and latidx1 != idx1[len(idx1)-1, 0]:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lon[i+1], lat[i+1]]], axis=0)
                lk = len(loc1)-1
                dist = geodesic((loc1[lk-1, 1], loc1[lk-1, 0]),
                                (loc1[lk, 1], loc1[lk, 0])).km
                dis1 = np.append(dis1, dist, axis=None)
                    
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(idx1)
    print(loc1)
    print(dis1)
    np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f') 
    np.savetxt("simu/test_dis.csv", dis1, delimiter=",", fmt='%f') 


72370 27617
m, b: -0.036292835590407094 29.481055103775862
line:  72371 121.54583333333335 0.5447916666666818 121.54479166666668 25.069849962947245 27617
line:  72372 121.55000000000001 0.5489583333333447 121.54895833333335 25.069698742798952 27617
line:  72373 121.55416666666667 0.5531250000000075 121.55312500000001 25.06954752265066 27617
line:  72374 121.55833333333334 0.5572916666666704 121.55729166666667 25.069396302502366 27617
line:  72375 121.5625 0.5614583333333333 121.56145833333333 25.069245082354072 27617
Exe time:  0.002085447311401367 sec
[[0 0]
 [0 1]
 [0 2]
 [0 3]
 [0 4]
 [0 5]
 [0 6]]
[[121.53967066  25.07003582]
 [121.54479167  25.06984996]
 [121.54895833  25.06969874]
 [121.553125    25.06954752]
 [121.55729167  25.0693963 ]
 [121.56145833  25.06924508]
 [121.56499244  25.06911682]]
[0.         0.51708303 0.42072113 0.42072165 0.42072217 0.42072268
 0.35685147]


In [ ]:
st = time.time()
#mlon0 = np.min(lon) #may cause slice offset to mlonbase, an offset +-1
mlon1 = np.max(lon) + 1.5/arc #to make it larger
#mlat0 = np.min(lat) #may cause slice offset to mlatbase, an offset +-1
mlat1 = np.max(lat) + 1.5/arc
mlon0x= ds["lon"][mlonbase]
mlat0x= ds["lat"][mlatbase]
ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(mlat0x, mlat1)) if subsetFlag else ds
print(ds_s1['elevation'].shape)
#mlonidx0 = gridded_arcsec(mlon0, basex, arc)
#mlatidx0 = gridded_arcsec(mlat0, basey, arc)
#mlonidx1 = gridded_arcsec(mlon1, basex, arc)
#mlatidx1 = gridded_arcsec(mlat1, basey, arc)
loct = tuple(idx1.T)
print(loct)
pt1=ds_s1['elevation'].values[loct]
et = time.time()
print('Get index of points, time: ', et-st, 'sec')
print(pt1)
np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

In [ ]:
ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1)) 
print(ds_s1)
print(ds["lat"][26720]) #actually more near 21.34, not 21.33 so slice 21.33 makes the offset 1 possibly!!
print(ds["lon"][72720]) #should use print(ds["lat"][26720]) to calcuate back
print(ds["elevation"].values[26720,72720]) 

In [ ]:
loc1[:,1]

In [ ]:
from fastapi.encoders import jsonable_encoder
#from fastapi.responses import JSONResponse
json_data = jsonable_encoder({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()})
#print(json_data)

import pandas as pd

df1 = pd.DataFrame({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()}, columns=['longitude', 'latitude', 'z'])
print(df1)

In [ ]:
from fastapi.encoders import jsonable_encoder
#just some  simple testing for query string
lax = "25.5, 30.22,20"
lox = " 125.33"
#la1 = lax.split(",")
if (',' in lax):
    try:
        la1 = [float(x.strip()) for x in lax.split(',')]
    except ValueError:
        print("Check your input format") 
else: 
    la1 = [float(lax.strip())]

if (',' in lox):
    lo1 = lox.split(",")
else: 
    lo1 = [float(lox.strip())]
print(la1)
print(lo1)

lon0 = gridded_arcsec(123, basex, arc)
lat0 = gridded_arcsec(21.33, basey, arc)
mlon0x = ds["lon"][lon0]
mlat0x = ds["lat"][lat0]
st1 = ds.sel(lon=mlon0x, lat=mlat0x)
xt1 = np.array([st1['elevation'].values])
print(xt1)
jt1 = {"data": xt1.tolist()}
jsonable_encoder(jt1)

In [ ]:
dis1 = np.array([0])
st = time.time()
for i in range(len(loc1)-1):
  dist = geodesic((loc1[i,1],loc1[i,0]), (loc1[i+1,1],loc1[i+1,0])).km
  dis1 = np.append(dis1, dist, axis=None)
et = time.time()
print('Calculate loop of geo dist time: ', et-st, 'sec')
print(dis1)

In [ ]:
#https://stackoverflow.com/questions/72620476/faster-way-to-calculate-distance-from-coordinates-on-dataframe
st = time.time()
df2 = pd.merge(df1.rename(columns={'latitude':'clat','longitude':'clon'}), df1, how='cross') \
            .assign(distance=lambda r: r.apply(lambda x: geodesic((x['clat'],x['clon']),(x['latitude'],x['longitude'])).km, axis=1)
            ) #.drop(columns=['clat','clon'])
et = time.time()
print('Calculate df of geo dist time: ', et-st, 'sec')
print(df2)

In [ ]:
# want to solve a situation when cross 180-line or zero-line (prime meridian)
l1a = [-179.95, 25]
l2a = [179,95, 24]
l1b = [-0.05, 25]
l2b = [0.05, 24]

print(geodesic((25,-179.95), (25,179.95)).km)
print(geodesic((25,-0.05), (25,0.05)).km)

def whichSide(l1a, l2a, direction='auto'):
   #posFlag = 'near-zero' #near prime meridian (near-zero) #otherwise, far-zero (near 180-degree)
   if l1a[0] != l2a[0] and np.sign(l1a[0]) != np.sign(l2a[0]):
      lonat1 = 180 + l1a[0]
      lonat2 = 180 + l2a[0]
      dift1 = np.absolute(lonat2 - lonat1)

      lonas1 = 180 - l1a[0] if l1a[0]>=0 else 180 - l2a[0]
      lonas2 = -180- l2a[0] if l2a[0]<0 else -180 - l1a[0]
      dift2 = np.absolute(lonas2 - lonas1) 

      if dift1 < dift2:
         posFlag = 'near-zero'
         print("use forward side", posFlag)
      else:
         posFlag = 'far-zero'
         print("use other side", posFlag)    

whichSide(l1a, l2a)
whichSide(l1b, l2b)
whichSide([-35.95, 25], [35, 24])
whichSide([-130.95, 25], [35, 24])
whichSide([-160.95, 25], [35, 24])


